In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import argparse
from scipy.stats import gaussian_kde
import os
import yaml
import h5py
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import time

from mpi_master_slave import Master, Slave
from mpi_master_slave import WorkQueue
from mpi4py import MPI
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
sys.path.append(r'../')
sys.path.append(r'../toollib_DisbaCode')
from objective_function import (ObjectiveFunctionDerivativeUsed, Forward)
from toollib_DisbaCode import forward
from toollib_DisbaCode import objective_function
from toollib_DisbaCode import empirical_relation
from toollib_DisbaCode import inversion
from toollib_DisbaCode import plot_disp
from toollib_DisbaCode import plot_model
from toollib_DisbaCode import plot_lcurve
from toollib_DisbaCode import plot_inversion    
from toollib_DisbaCode import plot_kernel

from toollib_standard import maplib
from toollib_standard import mathlib
#from toollib_standard import filelib
#from toollib_standard import stacklib
from toollib_standard import plotlib

In [ ]:
with open('a-project.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
proj_name = proj['name']
proj_name = 'project/output_FJSJ_14-01/'
proj_name

In [ ]:
filename = proj_name+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = proj_name+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
dir_ds = info_basic['dir_ds']
dir_image = info_basic['dir_image']
dir_inv_dispernet = info_basic['dir_inv_dispernet'] 
d_len = info_basic['d_len']
f = info_basic_bi['f']
c = np.linspace(info_basic['c_min'], info_basic['c_max'], info_basic['c_num'])
#c = info_basic_bi['c']

In [ ]:
key_subworks = info_basic['key_subworks']
#key_subworks = ['43-07']
key_subworks

### Forard and plot estimated disp and Vs

In [ ]:
flag_forward = 1

In [ ]:
stalistname = info_basic['stalistname']
stainfo = pd.read_excel(stalistname+'.xlsx')
nsta_all = len(stainfo.iloc[:,0])
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [ ]:
def plot_fj0(ax,ds_linear,title0,f,c,xlim,index,cl):
    ax.imshow(np.flip(ds_linear,0),aspect='auto',extent=[min(f),max(f),min(c),max(c)],vmin=0.05, cmap = 'jet',clim=cl)
    #plt.imshow(np.flip(ds_linear,0),extent=[min(f),max(f),min(c),max(c)],aspect='auto',cmap='jet',vmin=0,vmax=1)
    #ax.pcolormesh(f,c,ds_linear,cmap='jet',vmin=0,vmax=1)
    if index == 0:
        ax.set_title(title0)
    else:
        ax.set_title('('+chr(96+index)+')',loc='left')
    ax.set_xlabel('Normalized Frequency/ Hz')
    ax.set_ylabel('Velocity/ m/s')
    #ax.set_xlim(xlim)
    return ax

In [ ]:
def plot_fj(ax,ds_linear,title0,f,c,xlim,index,v_min,v_max,c_map):
    ax.imshow(np.flip(ds_linear,0),aspect='auto',extent=[min(f),max(f),min(c),max(c)],vmin=v_min,vmax = v_max, cmap = c_map)
    #plt.imshow(np.flip(ds_linear,0),extent=[min(f),max(f),min(c),max(c)],aspect='auto',cmap='jet',vmin=0,vmax=1)
    #ax.pcolormesh(f,c,ds_linear,cmap='jet',vmin=0,vmax=1)
    if index == 0:
        ax.set_title(title0)
    else:
        ax.set_title('('+chr(96+index)+')',loc='left')
    ax.set_xlabel('Normalized Frequency/ Hz')
    ax.set_ylabel('Velocity/ m/s')
    #ax.set_xlim(xlim)
    return ax

In [ ]:
"""
#c_map = info_basic['c_map']
#v_min = info_basic['v_min']
#v_max = info_basic['v_max']
c_map = 'jet'
v_max = None
v_min = 0.1


os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet')
file_config = 'config_inv.yml'
with open(file_config, 'r') as fp:
    config = yaml.safe_load(fp)

all_disp = 0
show_init = 0
xlim_f = [1,15]
ylim_c = [0.25,2.5]
clim = [0,1]

flag_num = 0
for key_subwork in key_subworks:
    fig,ax = plt.subplots(1,3,figsize=(27,8))

    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet/')
    data_file = 'data/ds_'+key_subwork+'curve.txt'
    file_model = 'model_'+key_subwork+'.txt'

    file_inv = 'inversion_files/ds_'+key_subwork+'curve/'
    if not os.path.exists(file_inv):
        print(key_subwork+'data file not exist')
        plt.close()
        continue



    disp_data = np.loadtxt(data_file)
    data_name = 'ds_'+key_subwork+'curve'
    ax[1] = plot_inversion.plot_disp(config, data_name, all_disp,ax[1])
    # s wave velocity inversion
    
    file_init = config['model_init']+key_subwork+'.txt'
    ax[2] = plot_inversion.plot_model(config, data_name, show_init, file_model,file_init,ax[2])
    

    # forward
    if flag_forward == 1:
        os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet/')
        max_mode = 3
        filename = 'disp_model_'+key_subwork+'.txt'
        forward.forward_cal(info_basic,max_mode,file_model,filename)


    disp_model = np.loadtxt(filename)


    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[0].set_xlim(xlim_f)
    ax[0].set_ylim(ylim_c)
    #ax[0] = plot_fj(ax[0],ds_remove,title0,f,c,xlim_f,0,clim)
    ax[0] = plot_fj(ax[0],ds_remove,title0,f,c,xlim_f,0,v_min,v_max,c_map)
    
    

    
    modes = set(disp_data[:, 2].astype(int))
    for m in modes:
        d = disp_data[disp_data[:, 2] == m]
        if m==0:
            ax[0].plot(d[:, 0], d[:, 1], 'g^',label='data')
        else:
            ax[0].plot(d[:, 0], d[:, 1], 'g^')

    modes = set(disp_model[:, 2].astype(int))
    for m in modes:
        d = disp_model[disp_model[:, 2] == m]
        if m==0:
            ax[0].plot(d[:, 0], d[:, 1], 'r--',label='estemate',linewidth=5)
        else:
            ax[0].plot(d[:, 0], d[:, 1], 'r--',linewidth=5)
    ax[0].legend()
    ax[2].set_title(key_subwork)


    plt.tight_layout()
    plt.savefig(dir_image+'inversion_'+key_subwork+'.png',dpi=60)
    plt.close()
    flag_num += 1
    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))
"""

In [ ]:
"""
os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
#key_subworks = ['12-04']
for key_subwork in key_subworks:
    fmin = 1
    fmax = 15
    cmin = 0.1
    cmax = 2.5
    vmax = 0.3
    
    file_disp = info_basic['dir_inv_dispernet']+'data/ds_'+key_subwork+'curve.txt'
    file_model = info_basic['dir_inv_dispernet']+'model_'+key_subwork+'.txt'
    disp = np.loadtxt(file_disp)
    m = int(max(set(disp[:,2])))

    fig,ax = plt.subplots(1,m+1,figsize=(7*(m+1),6))
    for mode in range(m+1):
        ax[mode] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,mode,vmax,ax[mode])
    #ax[0][1] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,1,vmax,ax[0][1])
    #ax[1][0] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,2,vmax,ax[1][0])
    #ax[1][1] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,3,vmax,ax[1][1])
    plt.tight_layout()
    plt.savefig(dir_image+'Sens-kernel_'+key_subwork+'.png',dpi=60)
    plt.close()
"""

In [ ]:
#key_subworks = ['25-06']
c_map = 'jet'
v_max = None
v_min = 0.1

os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet')
file_config = 'config_inv.yml'
with open(file_config, 'r') as fp:
    config = yaml.safe_load(fp)

all_disp = 0
show_init = 0
xlim_f = [1,30]
ylim_c = [0.25,1.8]
clim = [0,1]

flag_num = 0
for key_subwork in key_subworks:
    print(key_subwork)
    fig,ax = plt.subplots(3,3,figsize=(27,20))

    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet/')
    data_file = 'data/ds_'+key_subwork+'curve.txt'
    file_model = 'model_'+key_subwork+'.txt'

    file_inv = 'inversion_files/ds_'+key_subwork+'curve/'
    if not os.path.exists(file_inv):
        print(key_subwork+'data file not exist')
        plt.close()
        continue



    disp_data = np.loadtxt(data_file)
    data_name = 'ds_'+key_subwork+'curve'
    ax[1][1] = plot_inversion.plot_disp(config, data_name, all_disp,ax[1][1])
    # s wave velocity inversion
    
    file_init = config['model_init']+key_subwork+'.txt'
    ax[1][2] = plot_inversion.plot_model(config, data_name, show_init, file_model,file_init,ax[1][2])
    

    # forward
    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet/')
    filename = 'disp_model_'+key_subwork+'.txt'
    if flag_forward == 1:
        
        max_mode = 3
        
        forward.forward_cal(info_basic,max_mode,file_model,filename)


    disp_model = np.loadtxt(filename)


    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[1][0].set_xlim(xlim_f)
    ax[1][0].set_ylim(ylim_c)
    #ax[0] = plot_fj(ax[0],ds_remove,title0,f,c,xlim_f,0,clim)
    ax[1][0] = plot_fj(ax[1][0],ds_remove,title0,f,c,xlim_f,0,v_min,v_max,c_map)
    
    

    
    modes = set(disp_data[:, 2].astype(int))
    for m in modes:
        d = disp_data[disp_data[:, 2] == m]
        if m==0:
            ax[1][0].plot(d[:, 0], d[:, 1], 'g^',label='data')
        else:
            ax[1][0].plot(d[:, 0], d[:, 1], 'g^')

    modes = set(disp_model[:, 2].astype(int))
    for m in modes:
        d = disp_model[disp_model[:, 2] == m]
        if m==0:
            ax[1][0].plot(d[:, 0], d[:, 1], 'r--',label='estemate',linewidth=5)
        else:
            ax[1][0].plot(d[:, 0], d[:, 1], 'r--',linewidth=5)
    ax[1][0].legend()
    ax[1][2].set_title(key_subwork)


    #plt.tight_layout()
    #plt.savefig(dir_image+'inversion_'+key_subwork+'.png',dpi=60)
    #plt.close()
    flag_num += 1
    

    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
    fmin = 1
    fmax = 30
    cmin = 0.1
    cmax = 2.5
    vmax = 0.3
    
    file_disp = info_basic['dir_inv_dispernet']+'data/ds_'+key_subwork+'curve.txt'
    file_model = info_basic['dir_inv_dispernet']+'model_'+key_subwork+'.txt'
    disp = np.loadtxt(file_disp)
    m = int(max(set(disp[:,2])))
    
    #fig,ax = plt.subplots(1,m+1,figsize=(7*(m+1),6))
    if m > 2:
        m = 2
    for mode in range(m+1):
        #print(mode)
        try:
            ax[2][mode] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,mode,vmax,ax[2][mode])
        except:
            print('error')
    #ax[0][1] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,1,vmax,ax[0][1])
    #ax[1][0] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,2,vmax,ax[1][0])
    #ax[1][1] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,3,vmax,ax[1][1])


    xlim_T = [-1,1]
    start = info_basic_bi['start'][key_subwork]
    interval = info_basic_bi['interval'][key_subwork]
    flag_time = info_basic_bi['flag_time'][key_subwork]
    # plot area
    stalistname = proj_name+info_basic['stalistname']
    stainfo = pd.read_excel(stalistname+'.xlsx',key_subwork)
    nsta = len(stainfo.iloc[:,0])
    #StationPairs = GetStationPairs(nsta)
    #nPairs = int(len(StationPairs)/2)
    stalist = stainfo['Station'].tolist()
    lat = stainfo['latitude'].tolist() 
    lon = stainfo['longitude'].tolist()
    ax[0][0]=plotlib.plot_area(ax[0][0],lon_all,lat_all,lon,lat)


    # plot ncfs and ncfstW
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_linear = ds['ds_linear'][0]
    ds_remove = ds['ds_remove'][0]
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[0][1] = plotlib.plot_fj(ax[0][1],ds_linear,title0,f,c,xlim_f,0,v_min,v_max,c_map)
    ax[0][1].set_xlim(xlim_f)

    title0 = "Remove stack dispersion curve "+str(d_len)+' days'
    ax[0][2] = plotlib.plot_fj(ax[0][2],ds_remove,title0,f,c,xlim_f,0,v_min,v_max,c_map)
    ax[0][2].set_xlim(xlim_f)



    plt.tight_layout()
    #plt.savefig(dir_image+'Sens-kernel_'+key_subwork+'.png',dpi=60)
    plt.savefig(dir_image+'invplot_all_'+key_subwork+'.png',dpi=100)
    

    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))
    plt.close()

### Write into xlsx

In [ ]:
"""
# disp model xlsx
os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
filename = 'disp_model.xlsx'

if not os.path.exists(info_basic['dir_inv_dispernet']+filename):
    df = pd.DataFrame()
    df.to_excel(info_basic['dir_inv_dispernet']+filename,index=False)

books = load_workbook(info_basic['dir_inv_dispernet']+filename)
writers = pd.ExcelWriter(info_basic['dir_inv_dispernet']+filename, engine='openpyxl')
writers.book = books

flag_num = 0
for key_subwork in key_subworks:
    file_model = dir_inv_dispernet + 'disp_model_'+key_subwork+'.txt'
    disp_model = np.loadtxt(file_model)
    
    
    df = pd.DataFrame()
    df['f'] = disp_model[:, 0]
    df['c'] = disp_model[:, 1]
    df['mode'] = disp_model[:, 2]

    df.to_excel(writers,key_subwork,index=False)

    flag_num += 1
    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))

writers.save()
"""

In [ ]:

# disp data xlsx
os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
filename = 'structure_model.xlsx'

if not os.path.exists(info_basic['dir_inv_dispernet']+filename):
    df = pd.DataFrame()
    df.to_excel(info_basic['dir_inv_dispernet']+filename,index=False)

books = load_workbook(info_basic['dir_inv_dispernet']+filename)
writers = pd.ExcelWriter(info_basic['dir_inv_dispernet']+filename, engine='openpyxl')
writers.book = books

flag_num = 0
for key_subwork in key_subworks:
    file_model = dir_inv_dispernet + 'model_'+key_subwork+'.txt'
    model = np.loadtxt(file_model)
    
    
    df = pd.DataFrame()
    df['layer'] = model[:, 0]
    df['z'] = model[:, 1]
    df['rho'] = model[:, 2]
    df['vs'] = model[:, 3]
    df['vp'] = model[:, 4]
    #df['std'] = model[:, 5]
    df.to_excel(writers,key_subwork,index=False)

    flag_num += 1
    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))

writers.save()


In [ ]:
"""
# disp data xlsx
os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
filename = 'disp_data.xlsx'

if not os.path.exists(info_basic['dir_inv_dispernet']+filename):
    df = pd.DataFrame()
    df.to_excel(info_basic['dir_inv_dispernet']+filename,index=False)

books = load_workbook(info_basic['dir_inv_dispernet']+filename)
writers = pd.ExcelWriter(info_basic['dir_inv_dispernet']+filename, engine='openpyxl')
writers.book = books

flag_num = 0
for key_subwork in key_subworks:
    file_model = dir_inv_dispernet + '/data/ds_'+key_subwork+'curve.txt'
    disp_model = np.loadtxt(file_model)
    
    
    df = pd.DataFrame()
    df['f'] = disp_model[:, 0]
    df['c'] = disp_model[:, 1]
    df['mode'] = disp_model[:, 2]

    df.to_excel(writers,key_subwork,index=False)

    flag_num += 1
    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))

writers.save()
"""

In [ ]:
"""
dir_inv_dispernet = info_basic['dir_inv_dispernet']
os.listdir(dir_inv_dispernet)[0][-3:]
for file in os.listdir(dir_inv_dispernet):
    if file[-3:] == 'txt':
        os.remove(dir_inv_dispernet+file)
"""